In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
     |████████████████████████████████| 120 kB 51.5 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.4 MB/s 
     |████████████████████████████████| 115 kB 24.9 MB/s 
     |████████████████████████████████| 212 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import torch
import time
from torch.utils.data import Dataset, DataLoader
from torch import nn
import random
import numpy as np
from typing import Dict
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
#fix random seed
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load our dataset
gap_dataset = load_dataset("gap")

# let's instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/454 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset gap downloaded and prepared to /root/.cache/huggingface/datasets/gap/default/0.1.0/f0e9cd15a173714ee6447d210f68779fcd7e4c90e2f1921b7ecf78e5aeead18a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def find_max(data):
  max = 0
  for i in data:
    if len(i) > max:
      max = len(i)
  return max

In [ ]:
#transform data so each phrase has the same length
def all_same_length_bert(embeds, token_id, attention_mask, labels, max):
  for i in range(0, len(embeds)):
    diff = max - len(embeds[i]) 
    if diff > 0:
      for x in range(0, diff):
        embeds[i].append(0)
        token_id[i].append(0)
        attention_mask[i].append(0)
        labels[i].append(0)

  return embeds, token_id, attention_mask, labels

In [ ]:
#encode words with tokenizer keep traking

def tokenize_words(data):
  #lista = ['her', 'his', 'she', 'he', 'him'] #there are only these pronouns because are the only ones present on training_set
  labels = []
  embeddings = []
  token_id = []
  attention_mask = []
  tot_count = []
  tot_position = []
  for i in data:
    phrase = i['Text']
    word_list = phrase.split()
    offset = i['Pronoun-offset']
    count = 0
    conto = []
    pos = 0
    posizione = []
    ##posizione = {}
    ##c = 0
    
    lab = [0]
    embed = [101]
    att = [1]
    token = [0]

    for j in word_list:
      #save count characters
      conto.append(count)

      #tokenize word
      batch_out = tokenizer(j, padding=True, truncation=True)
      
      #update position
      pos = (len(batch_out['input_ids']) - 2)
      posizione.append(pos)

      ##posizione[pos] = c
      ##c += 1
      #####pos += (len(batch_out['input_ids']) - 2)

      #set label
      if count == offset:
        l = 1
      else:
        l = 0

      #save embeddings and labels
      for i in range(1, len(batch_out['input_ids'])-1):
        embed.append(batch_out['input_ids'][i])
        token.append(0)
        att.append(1)
        lab.append(l)
      
      #update count characters
      count += len(j) + 1
      
    #embed = torch.FloatTensor(embed)
    embed.append(102)
    token.append(0)
    att.append(1)
    lab.append(0)
    embeddings.append(embed)
    token_id.append(token)
    attention_mask.append(att)
    labels.append(lab)
    tot_count.append(conto)
    ##posizione.append(pos)#########
    tot_position.append(posizione)
  return embeddings, token_id, attention_mask, labels, tot_count, tot_position

In [ ]:
#create embeds and labels
embeds_train, token_id_train, attention_mask_train, labels_train, tot_count_train, tot_pos_train = tokenize_words(gap_dataset['train'])
embeds_valid, token_id_valid, attention_mask_valid, labels_valid, tot_count_valid, tot_pos_valid = tokenize_words(gap_dataset['validation'])
embeds_test, token_id_test, attention_mask_test, labels_test, tot_count_test, tot_pos_test = tokenize_words(gap_dataset['test'])

In [ ]:
#create all data of the same length
embeds_train, token_id_train, attention_mask_train, labels_train = all_same_length_bert(embeds_train, token_id_train, attention_mask_train, labels_train, find_max(embeds_train))
embeds_valid, token_id_valid, attention_mask_valid, labels_valid = all_same_length_bert(embeds_valid, token_id_valid, attention_mask_valid, labels_valid, find_max(embeds_valid))
embeds_test, token_id_test, attention_mask_test, labels_test = all_same_length_bert(embeds_test, token_id_test, attention_mask_test, labels_test, find_max(embeds_test))

In [ ]:
def collate_fn(embeds, labels):
  data = {}
  #embeds, token_id, attention_mask, labels, _, _ = tokenize_words(batch)
  #embeds, token_id, attention_mask, labels = all_same_length_bert(embeds, token_id, attention_mask, labels, find_max(embeds))
  
  for i in range(0, len(embeds)):
    data[i]={'inputs': torch.LongTensor(embeds[i]), 'outputs': torch.LongTensor(labels[i])}

  return data

In [ ]:
data_train = collate_fn(embeds_train, labels_train)
data_valid = collate_fn(embeds_valid, labels_valid)
data_test = collate_fn(embeds_test, labels_test)

In [ ]:
train_dataloader = DataLoader(data_train, batch_size=64, shuffle=True)

valid_dataloader = DataLoader(data_valid, batch_size=64, shuffle=False,)

test_dataloader = DataLoader(data_test, batch_size=64, shuffle=False,)

In [ ]:
###
train_dataloader = DataLoader(
    gap_dataset["train"],
    batch_size=64,
    collate_fn=collate_fn,
    shuffle=True,
)

valid_dataloader = DataLoader(
    gap_dataset["validation"],
    batch_size=64,
    collate_fn=collate_fn,
    shuffle=False,
)

test_dataloader = DataLoader(
    gap_dataset["test"],
    batch_size=64,
    collate_fn=collate_fn,
    shuffle=False,
)

In [ ]:
bert = AutoModel.from_pretrained('bert-base-cased', output_hidden_states=True)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#class params to set all the hyperparameters
class HParams_bilstm():
    vocab_size = tokenizer.vocab_size
    hidden_dim = 128
    embedding_dim = 768
    num_classes = 2
    bidirectional = True
    num_layers = 1
    dropout = 0.005
    embeddings = bert.embeddings.word_embeddings.weight
    #class_vector = []
    
params = HParams_bilstm()

In [ ]:
#create model
class BiLstm_pronouns(nn.Module):

    def __init__(self, hparams):
        super(BiLstm_pronouns, self).__init__()
        self.word_embedding = nn.Embedding(hparams.vocab_size, hparams.embedding_dim)
        self.word_embedding.weight.data.copy_(hparams.embeddings)

        self.lstm = nn.LSTM(hparams.embedding_dim, hparams.hidden_dim, 
                            bidirectional=hparams.bidirectional,
                            num_layers=hparams.num_layers,
                            batch_first = True,
                            dropout = hparams.dropout if hparams.num_layers > 1 else 0)
        
        lstm_output_dim = hparams.hidden_dim * 2
    
        self.fc1 = nn.Linear(lstm_output_dim, 64)

        self.dropout = nn.Dropout(hparams.dropout)

        self.classifier = nn.Linear(64, hparams.num_classes)

    
    def forward(self, x):
        embeddings = self.word_embedding(x)
        embeddings = self.dropout(embeddings)
        x, (h, c) = self.lstm(embeddings)
        x = self.dropout(x)
        o = self.fc1(x)
        o = self.dropout(o)
        

        output = self.classifier(o)
        return output

In [ ]:
class Trainer():
    """Utility class to train and evaluate a model."""

    def __init__(
        self,
        model: nn.Module,
        loss_function,
        optimizer,
        #label_vocab: indices_to_label,
        log_steps:int=10_000,
        log_level:int=2):
        """
        Args:
            model: the model we want to train.
            loss_function: the loss_function to minimize.
            optimizer: the optimizer used to minimize the loss_function.
        """
        self.model = model
        self.loss_function = loss_function
        self.optimizer = optimizer

        #self.label_vocab = label_vocab
        self.log_steps = log_steps
        self.log_level = log_level
        #self.label_vocab = label_vocab

    def train(self, train_dataset:Dataset, 
              valid_dataset:Dataset, 
              epochs:int=1):

        since = time.time()
        since2 = since
        train_loss = 0.0
      
        assert epochs > 1 and isinstance(epochs, int)
        '''if self.log_level > 0:
            print('Training ...')'''
        train_loss = 0.0
        for epoch in range(epochs):
            if self.log_level > 0:
                print('Epoch: ', epoch)

            epoch_loss = 0.0
            self.model.train()

            # for each batch 
            for step, sample in enumerate(train_dataset):
                inputs = sample['inputs']
                labels = sample['outputs']
                self.optimizer.zero_grad()

                predictions = self.model(inputs)
                predictions = predictions.view(-1, predictions.shape[-1])
                labels = labels.view(-1)
                # labels  [[1,2,3], [18, 12, 3]] after the view(-1) [1,2,3, 18, 12, 3]
                
                sample_loss = self.loss_function(predictions, labels)
                sample_loss.backward()
                self.optimizer.step()

                epoch_loss += sample_loss.tolist()

                '''if self.log_level > 1 and step % self.log_steps == self.log_steps - 1:
                    print('\t[E: {:2d} @ step {}] current avg loss = {:0.4f}'.format(epoch, step, epoch_loss / (step + 1)))'''
            
            avg_epoch_loss = epoch_loss / len(train_dataset)
            train_loss += avg_epoch_loss

            print('Epoch loss: ', epoch_loss, ' || Average epoch loss: ', avg_epoch_loss)
            time_elapsed = time.time() - since2
            since2 = time.time()
            print('Time to complete epochs {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

            valid_loss = self.evaluate(valid_dataset)
            print('Validation loss: ', valid_loss)

            print('---------------------------------------')
            
            '''if self.log_level > 0:
                print('  [E: {:2d}] valid loss = {:0.4f}'.format(epoch, valid_loss))'''

        '''if self.log_level > 0:
            print('... Done!')'''
        
        print('-----end-----')
        avg_epoch_loss = train_loss / epochs
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Average epoch loss: ', avg_epoch_loss)
        
        return avg_epoch_loss
    

    def evaluate(self, valid_dataset):
        """
        Args:
            valid_dataset: the dataset to use to evaluate the model.

        Returns:
            avg_valid_loss: the average validation loss over valid_dataset.
        """
        valid_loss = 0.0
        # set dropout to 0!! Needed when we are in inference mode.
        self.model.eval()
        with torch.no_grad():
            for sample in valid_dataset:
                inputs = sample['inputs']
                labels = sample['outputs']

                predictions = self.model(inputs)
                pred = predictions.view(-1, predictions.shape[-1])
                lab = labels.view(-1)
                

                '''#print('PREDICTIONS: ', predictions.size())
                #l = torch.argmax(predictions, -1)
                l = torch.argmax(predictions, -1, keepdim=True)
                l = torch.argmax(l, 1, keepdim=True)
                print('L: ', l.size())
                print('L: ', l)
                pred = predictions.view(-1, predictions.shape[-1])
                #print('PRED: ', pred.size())
                lab = labels.view(-1)
                #print('LAB: ', lab.size())
                #print('LABRLS: ', labels.tolist())
                top_label_scores, top_label_indices = torch.max(pred, -1)
                #print('top_label_scores: ', top_label_scores.size())
                #print('top_label_indices: ', top_label_indices.size())
                top_label_indices = top_label_indices.tolist()
                #print('PREDICTION: ', top_label_indices)'''


                sample_loss = self.loss_function(pred, lab)
                valid_loss += sample_loss.tolist()
                #print('sample_loss: ', sample_loss)
                #print('valid_loss: ', valid_loss)
                #f1_score1 = f1_score(lab.data, top_label_indices, average="macro")
                #print('f1: ', f1_score1)

        
        return valid_loss / len(valid_dataset)

    def predict(self, x):

        self.model.eval()
        with torch.no_grad():
            logits = self.model(x)
            predictions = torch.argmax(logits, -1)
            #print(predictions)
            return logits, predictions

In [ ]:
bilstm_embed_classifier = BiLstm_pronouns(params)#.cuda()

In [ ]:
bilstm_trainer = Trainer(
    model = bilstm_embed_classifier,
    loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1/1000, 1])), #204
    optimizer = torch.optim.Adam(bilstm_embed_classifier.parameters()), #lr=0.006
)

In [ ]:
bilstm_trainer.train(train_dataloader, valid_dataloader, 6)
PATH = '/content/drive/MyDrive/Gap_dataset/model_tokenizer_1.pt'

torch.save(bilstm_embed_classifier, PATH)

Epoch:  0
Epoch loss:  7.656473197042942  || Average epoch loss:  0.23926478740759194
Time to complete epochs 1m 30s
Validation loss:  0.06656005838885903
---------------------------------------
Epoch:  1
Epoch loss:  0.7310519879683852  || Average epoch loss:  0.022845374624012038
Time to complete epochs 1m 38s
Validation loss:  0.030771765625104308
---------------------------------------
Epoch:  2
Epoch loss:  0.4066159147769213  || Average epoch loss:  0.01270674733677879
Time to complete epochs 1m 34s
Validation loss:  0.034202240873128176
---------------------------------------
Epoch:  3
Epoch loss:  0.2921702931635082  || Average epoch loss:  0.00913032166135963
Time to complete epochs 1m 33s
Validation loss:  0.10857665282674134
---------------------------------------
Epoch:  4
Epoch loss:  0.20846893033012748  || Average epoch loss:  0.006514654072816484
Time to complete epochs 1m 32s
Validation loss:  0.09391636541113257
---------------------------------------
Epoch:  5
Epoch 

In [ ]:
PATH = '/content/drive/MyDrive/Gap_dataset/model_tokenbizer_1.pt'
model = torch.load(PATH)
model.eval()

bilstm_trainer = Trainer(
    model = model,
    loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1/1000, 1])), #204
    optimizer = torch.optim.Adam(model.parameters()), #lr=0.006
)

In [ ]:
#decoder of the results

def prediction(embeds, tot_count, tot_pos):
  pron = [1123, 1117, 1131, 1119, 1140, 1430, 1230, 1153, 1124, 15619]
  for i in range(0, len(embeds)):
    logits, predict = bilstm_trainer.predict(torch.LongTensor(embeds[i]))
    index = -1
    max = 0
    for j in range(0, predict.size(dim=0)):
      pre = predict[j].numpy()
      if pre == 1:
        if embeds[i][j] in pron:
          class_1 = logits[j][1].item()
          #print('score: ', class_1)
          if class_1 > max:
            max = class_1
            index = j
    tot = 0
    count_index = 0
    for p in range(0, len(tot_pos_test[i])):
      tot += tot_pos_test[i][p]
      if tot > index:
        count_index = p - 1
        break
    #print(predict)
    #print('max: ', max)
    print('embeds: ', embeds_test[i][index: index+ tot_pos_test[i][count_index]])
    print('word: ', tokenizer.decode(embeds_test[i][index: index+ tot_pos_test[i][count_index]]))
    print('character: ', tot_count_test[i][count_index])

    print('##Correct##')
    print('Word: ', gap_dataset['test']['Pronoun'][i])
    print('char: ', gap_dataset['test']['Pronoun-offset'][i])
    if i == 10:
      return

      
    print('--------------------------')

In [ ]:
prediction(embeds_test, tot_count_test, tot_pos_test)

embeds:  [1230]
word:  His
character:  383
##Correct##
Word:  His
char:  383
--------------------------
embeds:  [1117]
word:  his
character:  537
##Correct##
Word:  him
char:  430
--------------------------
embeds:  [1124]
word:  He
character:  312
##Correct##
Word:  He
char:  312
--------------------------
embeds:  [1117]
word:  his
character:  526
##Correct##
Word:  his
char:  526
--------------------------
embeds:  [1117]
word:  his
character:  406
##Correct##
Word:  his
char:  406
--------------------------
embeds:  [1123]
word:  her
character:  349
##Correct##
Word:  her
char:  349
--------------------------
embeds:  [1153]
word:  She
character:  365
##Correct##
Word:  She
char:  365
--------------------------
embeds:  [1123]
word:  her
character:  299
##Correct##
Word:  her
char:  307
--------------------------
embeds:  [1117]
word:  his
character:  359
##Correct##
Word:  he
char:  227
--------------------------
embeds:  [1123]
word:  her
character:  223
##Correct##
Word:  her
c

In [ ]:
print(gap_dataset['test'][8])

{'ID': 'test-9', 'Text': 'On June 4, 1973 at the Felt Forum, Madison Square Garden, New York, Malave lost to Ray Lampkin by an eighth-round knockout. Malave took a fight in Boston, Mass. against Greg Joiner, winning by a knockout in the 3rd round. Then he faced former World Lightweight Champion Ken Buchanan, losing by a TKO in the 7th round September 1, 1973, which turned out to be his last professional boxing match.', 'Pronoun': 'he', 'Pronoun-offset': 227, 'A': 'Malave', 'A-offset': 124, 'A-coref': True, 'B': 'Greg Joiner', 'B-offset': 169, 'B-coref': False, 'URL': 'http://en.wikipedia.org/wiki/Edwin_Malave'}


In [ ]:
def prediction_check(embeds, tot_count, tot_pos):
  pron = [1123, 1117, 1131, 1119, 1140, 1430, 1230, 1153, 1124, 15619]
  ap = 0
  apc = 0
  for i in range(0, len(embeds)):
    logits, predict = bilstm_trainer.predict(torch.LongTensor(embeds[i]))
    index = -1
    max = 0
    for j in range(0, predict.size(dim=0)):
      pre = predict[j].numpy()
      if pre == 1:
        if embeds[i][j] in pron:
          class_1 = logits[j][1].item()
          #print('score: ', class_1)
          if class_1 > max:
            max = class_1
            index = j
    tot = 0
    count_index = 0
    for p in range(0, len(tot_pos_test[i])):
      tot += tot_pos_test[i][p]
      if tot > index:
        count_index = p - 1
        break
    #print(predict)
    #print('max: ', max)
    #print('embeds: ', embeds_test[i][index: index+ tot_pos_test[i][count_index]])
    #print('word: ', tokenizer.decode(embeds_test[i][index: index+ tot_pos_test[i][count_index]]))
    #print('character: ', tot_count_test[i][count_index])
    if tokenizer.decode(embeds_test[i][index: index+ tot_pos_test[i][count_index]]) == gap_dataset['test']['Pronoun'][i]:
      ap += 1
    if tot_count_test[i][count_index] == gap_dataset['test']['Pronoun-offset'][i]:
      apc += 1
    

    #print('##Correct##')
    #print('Word: ', gap_dataset['test']['Pronoun'][i])
    #print('char: ', gap_dataset['test']['Pronoun-offset'][i])
      
  print('Number of right ap detected: ', ap)
  print('Number of right position of ap detected: ', apc)
  print('Tot phrases examinated: ', len(gap_dataset['test']))

In [ ]:
prediction_check(embeds_test, tot_count_test, tot_pos_test)

Number of right ap detected:  1448
Number of right position of ap detected:  1169
Tot phrases examinated:  2000


In [ ]:
logits, predict = bilstm_trainer.predict(torch.LongTensor(embeds_test[0]))
#print(logits)
index = -1
max = 0
for i in range(0, predict.size(dim=0)):
  pre = predict[i].numpy()
  if pre == 1:
    print('predictions: ', pre)
    print('index: ', i)
    #class_0 = logits[i][0].item()
    class_1 = logits[i][1].item()
    print('logits: ', class_1)
    #print('logits: ', class_0, class_1)
    if class_1 > max:
      index = i
    try:
      print('count: ', tot_count_test[0][tot_pos_test[0][i]])
      print('tot_pos: ', tot_pos_test[0][i])
      print('frase: ', embeds_test[0][i: i+(tot_pos_test[0][i+1]- tot_pos_test[0][i])])
      print('parola: ', tokenizer.decode(embeds_test[0][i: i+(tot_pos_test[0][i+1]- tot_pos_test[0][i])]))
    except:
      pass
print('--------------------------')

print('Index AP: ', index)
print('count: ', tot_count_test[0])
print('tot_pos: ', tot_pos_test[0])

print(gap_dataset['test']['Text'][0][307:])
print(gap_dataset['test'][0])

#for i in range(0, len(labels_test[0])):
#  print(embeds_test[0][i], ' | ', labels_test[0][i], ' | ' , i)

print('parola: ', tokenizer.decode(embeds_test[0][98]))








#token_id_test, attention_mask_test, labels_test
#tot_count_test, tot_pos_test

predictions:  1
index:  55
logits:  2.753734588623047
predictions:  1
index:  98
logits:  2.6470179557800293
--------------------------
Index AP:  98
count:  [0, 5, 11, 22, 27, 31, 43, 50, 58, 65, 70, 78, 81, 86, 88, 97, 100, 108, 113, 117, 125, 128, 132, 136, 139, 144, 148, 154, 160, 170, 174, 181, 194, 199, 203, 211, 216, 219, 225, 228, 233, 237, 242, 246, 250, 257, 260, 263, 270, 274, 279, 283, 293, 296, 301, 307, 314, 320, 325, 331, 335, 352, 356, 362, 366, 375, 383, 387, 394, 397, 407, 414, 424, 432, 437]
tot_pos:  [0, 1, 2, 3, 4, 5, 8, 9, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 29, 30, 31, 33, 34, 35, 37, 39, 40, 41, 42, 43, 44, 45, 46, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 69, 71, 72, 73, 75, 76, 78, 79, 81, 82, 85, 86, 88, 89, 94, 97, 98, 99, 100, 101, 104, 105, 106, 107, 110]
Former NHLer Gary Suter and Olympic-medalist Bob Suter are Dehner's uncles. His cousin is Minnesota Wild's alternate captain Ryan Suter.
{'ID': 'test-1', 'Text': "Upon th

In [ ]:
tokens = ['her', 'his', 'she', 'he', 'him', 'Her', 'His', 'She', 'He', 'Him' ]
ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
ids

[1123, 1117, 1131, 1119, 1140, 1430, 1230, 1153, 1124, 15619]